Fichier de test de fonction, aucune valeur pour le projet  
a supprimer à la fin du projet

In [1]:
import os
import io
import s3fs
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import numpy as np
import seaborn as sns

!pip install nltk
import nltk
nltk.download('punkt')
%matplotlib inline

#fichier projet
!pip install psycopg2
from bdd import *
!pip install Unidecode
from preprocessing import * 
!pip install pyenchant

/opt/conda/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Looking in indexes: https://nexus.beta.innovation.insee.eu/repository/pypi-public/simple


[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Looking in indexes: https://nexus.beta.innovation.insee.eu/repository/pypi-public/simple
Looking in indexes: https://nexus.beta.innovation.insee.eu/repository/pypi-public/simple
     |████████████████████████████████| 245kB 61.8MB/s eta 0:00:01
Looking in indexes: https://nexus.beta.innovation.insee.eu/repository/pypi-public/simple
     |████████████████████████████████| 61kB 64.9MB/s eta 0:00:01


In [2]:
my_driver = PostGre_SQL_DB()

In [3]:
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': 'http://minio.stable.innovation.insee.eu'})
remote_data_dir="s3://groupe-1033/data"
fs.ls(remote_data_dir)
with fs.open(os.path.join("s3://groupe-1033/data/",'settings.yml')) as f:
     settings = yaml.safe_load(f)

In [4]:
df = my_driver.read_from_sql(''' 
SELECT *
FROM rp_final_2017
WHERE 1=1
AND siretc IS NOT NULL
AND code_retour = 'C_C'
AND i_mca_c = 'C'
LIMIT 1; ''')
df.head()

,cabbi,typcol,depcom_code,rs_x,ilt_x,vardompart_x,numvoi_x,bister_x,typevoi_x,nomvoi_x,cpladr_x,actet_x,actet_c_c,actet_c,clt_c_c,clt_x,dlt_x,plt_x,siretc,note_mca_adr_c,note_mca_nom_c,i_mca_c,profi_x,profs_x,profa_x,i_reprise_act,i_siret_m,siret_dec,siretm,proces_codage_init,arbitrage,siretq,i_siret_q,siret_arb,x,y,sourcexyw,qual,siret_final,code_retour,naf1,pct1,naf2,pct2,naf3,pct3,naf4,pct4,naf5,pct5,nbtot,nb5
0,7300076772,None,03021,IEMTHESE AGEPAPH,2,None,0073,None,RTE,DE SAULCET,None,(CENTRE) INSTITUE D EDUCATION MOTRICE,8710B,8710B,03254,SAINT POURCAIN,03,None,49165388700043,10.0,466.0,C,None,AMP,None,None,None,49165388700043,None,None,None,None,None,None,721502.79,6579439.67,None,1.0,None,C_C,None,None,None,None,None,None,None,None,None,None,None,None


In [5]:
df = my_driver.read_from_sql(''' 
SELECT *
FROM sirus_2017
WHERE 1=1
AND SIRUS_ID = '491653887'
AND NIC = '00043'
 ''')
df.head()

,sirus_id,nic,ape,apet,eff_3112_et,eff_etp_et,eff_et_effet_daaaammjj,enseigne_et1,nom_comm_et,adr_et_loc_geo,adr_et_compl,adr_et_voie_num,adr_et_voie_repet,adr_et_voie_type,adr_et_voie_lib,adr_et_cedex,adr_et_distsp,sir_adr_et_com_lib,adr_et_post,adr_et_l1,adr_et_l2,adr_et_l3,adr_et_l4,adr_et_l5,adr_et_l6,adr_et_l7,nic_siege,unite_type,region,adr_depcom,region_impl,region_mult,tr_eff_etp,cj,denom,denom_condense,sigle,enseigne,eff_3112_unitelegale,eff_etp_unitelegale,eff_effet_daaaammjj_unitelegale,x,y,sourcexyw,qual
0,491653887,00043,8730A,8710B,51.0,48.0,20151231,None,None,0325400286,INSTITUT D EDUCATION MOTRICE,73,None,RTE,DE SAULCET,None,None,SAINT POURCAIN SUR SIOULE,03500,GESTION D ETS PERSONNES AGEES ...,None,INSTITUT D EDUCATION MOTRICE,73 RTE DE SAULCET,None,03500 SAINT POURCAIN SUR SIOULE,None,00092,1,84,03254,84,QASI,31,9220,GESTION D ETABLISSEMENTS POUR PERSONNES AGEES ...,GESTION D ETS PERSONNES AGEES ...,AGEPAPH,None,251.0,230.0,20151231,721502.79,6579439.67,None,1


In [6]:
df = my_driver.read_from_sql(''' 
SELECT DISTINCT(sir_adr_et_com_lib)
FROM sirus_2017
WHERE 1=1
AND sir_adr_et_com_lib LIKE '%%SAINT POURCAIN%%'
 ''')
df.head()

,sir_adr_et_com_lib
0,SAINT POURCAIN SUR BESBRE
1,SAINT POURCAIN SUR SIOULE


In [7]:
df = my_driver.read_from_sql(''' 
SELECT DISTINCT(sir_adr_et_com_lib)
FROM sirus_2017
UNION
SELECT DISTINCT(sir_adr_et_com_lib)
FROM sirus_2014
 ''')
df.head()

,sir_adr_et_com_lib
0,EPISY
1,ETH
2,LIZY
3,VERNAUX
4,SAUMANE


In [8]:
df.shape

(36121, 1)

In [9]:
def constitution_dictionnaire():
    df = my_driver.read_from_sql(''' 
    SELECT DISTINCT(sir_adr_et_com_lib)
    FROM sirus_2017
    UNION
    SELECT DISTINCT(sir_adr_et_com_lib)
    FROM sirus_2014
     ''')
    brique = []
    mots = df.sir_adr_et_com_lib.values
    for item in mots:
        brique.append(remove_punctuation(item))
    brique = " ".join(brique)
    mots = nltk.word_tokenize(brique)
    unique_mot = list(set(mots))
    return unique_mot

In [37]:
def correction_orthographe_commune(commune,dictionnaire):
    
    # Il faut une collection avec tout les noms de communes (sirus)
    # une fois récupéré, par exemple Saint-Fargo devient
    # ['SAINT','FARGO'] on peut ainsi vérifier mot par mot
    # Absolument récupéré les anciennes communes
    # Utiliser le numero de département si disponible ?
    
    mots_correcte = []
    mots_commune = commune.split(" ")
    #Heuristique : On prend le mot ayant la distance la plus courte
    # Que faire en cas d'égalité ??
    for mot in mots_commune:
        if mot in dictionnaire:
            mots_correcte.append(mot)
        else:
            sug = pwl.suggest(mot)
            if len(sug) == 1:
                mots_correcte.append(sug[0])
            else:
                print(str(len(sug)))
    correction = " ".join(mots_correcte)
    #Si on a un doute sur un mot, on ne corrige rien
    if len(mots_commune) != len(mots_correcte):
        return commune
    else:
        return correction

In [11]:
df = my_driver.read_from_sql(''' 
SELECT DISTINCT(sir_adr_et_com_lib)
FROM sirus_2017
UNION
SELECT DISTINCT(sir_adr_et_com_lib)
FROM sirus_2014
 ''')
list_communes = list(set(df.sir_adr_et_com_lib.values))

In [12]:
len(list_communes)

36121

In [46]:
df = my_driver.read_from_sql(''' 
SELECT DISTINCT(clt_x)
FROM rp_final_2017
WHERE clt_x IS NOT NULL
 ''')
communes_X = df.clt_x.values
communes_X = [trim(letters_only(x)) for x in communes_X]
df.head()

,clt_x
0,75009PARIS
1,SAUMANE
2,ESTERENCUBY
3,PIERREFITTE-NESTALAS
4,REGION PARISIENNEN ET NORMANDI


In [14]:
non_trouve = [x for x in communes_X if x not in list_communes]

In [15]:
len(communes_X)

60588

In [16]:
len(non_trouve)

26821

In [17]:
import enchant
import sys

In [26]:
dd = constitution_dictionnaire()

In [29]:
MyFile=open('dict.txt','w')

for element in dd:
    MyFile.write(element)
    MyFile.write('\n')
MyFile.close()

In [30]:
pwl = enchant.request_pwl_dict("dict.txt")

In [32]:
pwl.suggest("MASSYR")

['MASSY']

In [50]:
a = list(filter(None,communes_X))

In [ ]:
# communes_X_Y = [correction_orthographe_commune(c,dd) for c in a]

In [54]:
non_trouve_Y = [x for x in communes_X_Y if x not in list_communes]

In [55]:
len(non_trouve_Y)

23087

In [56]:
non_trouve_Y

['REGION PARISIENNEN ET NORMANDI',
 'CHAMPAGNE ARDENNES',
 'PROVINS HERME',
 'L ILE SAINT DENIS',
 'HH',
 'ROISSY AEROPORT CHARLES DE GAU',
 'SATROVILLE',
 'LES CLAYS SOUS BOIS',
 'SAINT DIDIER SUR CHALARD',
 'ST BLAISE',
 'BUSSIERES LES BELMONT',
 'APPRIEU COLOMBE VOIRON S',
 'FERIGNY',
 'PERPIGNAN SAINT LAURENT CANET',
 'SAINT OUEN L AUMONE',
 'MITTELWIHR',
 'JUSSIEU',
 'VILLENAVE D ORNON',
 'PARIS EME ARRONDISSEMENT',
 'FLASSENS',
 'FLEUY LES AUBRAIS',
 'VANNES LOCMINE AURAY',
 'NANTERRE VILLE',
 'VINCA ILLE TET BOULETERNERE',
 'SAINT GERMAIN TLLDE',
 'SCEAUX BOUR LA REINE ANTONY',
 'VARIABLE TOUS LES JOURS',
 'RIVE EN SCENE',
 'ATHIS MOS',
 'ERQINGHEM LYS',
 'ROZELAY',
 'LA PLEINE ST DENIS',
 'FRESNE LES MONTAUBAN',
 'VILLIERS EN PLAINES',
 'MEZZANA BOCOGNANO AJACCIO',
 'ROCHEFORT ROYAN',
 'SAINT AVOLD ET ARRONDISSEMENTS',
 'BOIRY STE RICTRUDE',
 'MIGNE AUXANCE',
 'SEVRE ANXAUMONT',
 'SAINT SALVY DE LABALME',
 'MONTRICHER',
 'SAVIGNY EN VERRON',
 'AY CHAMPAGNE',
 'ET',
 'VIRE NORMA

In [152]:
dd = constitution_dictionnaire()

'MASSY LYS'

In [175]:
# communes_X_Y = [correction_orthographe_commune(c,dd) for c in communes_X]

In [39]:
# communes_X[0:100]

In [20]:
len(purge_num)

39023

In [114]:
nltk.edit_distance('A-A', 'A A')

1

In [78]:
vector_distance = [(nltk.edit_distance('VLAETTE', x),x) for x in unique_mot]

In [79]:
sorted_by_second = sorted(vector_distance, key=lambda tup: tup[0])

In [135]:
print(correction_orthographe_commune('FLANCOURT CRESCY EN RUMOIS',dd))

['FLANCOURT', 'CRESCY', 'EN', 'RUMOIS']
FLANCOURT CRESCY EN ROUMOIS


In [81]:
# sorted_by_second

In [87]:
def dtypes_to_list(df):
    my_list = {}
    for x in df.dtypes.reset_index().iterrows():
        my_list[x[1]['index'].lower()] = str(x[1][0])
    return my_list

In [10]:
print(df2014.dtypes)

CABBI                   int64
TYPCOL                  int64
DEPCOM_CODE            object
RS_X                   object
ILT_X                 float64
VARDOMPART_X          float64
NUMVOI_X              float64
BISTER_X               object
TYPEVOI_X              object
NOMVOI_X               object
CPLADR_X               object
ACTET_X                object
ACTET_C_C              object
ACTET_C                object
CLT_C_C                object
CLT_X                  object
DLT_X                  object
PLT_X                 float64
SIRETC                float64
NOTE_MCA_ADR_C        float64
NOTE_MCA_NOM_C        float64
I_MCA_C                object
PROFI_X                object
PROFS_X                object
PROFA_X                object
I_REPRISE_ACT          object
I_SIRET_M             float64
SIRET_DEC               int64
SIRETM                float64
PROCES_CODAGE_INIT     object
ARBITRAGE              object
SIRETQ                float64
I_SIRET_Q               int64
SIRET_ARB 

In [12]:
print(df2017.dtypes)

CABBI             object
RS_X              object
ACTET_C           object
ACTET_C_C         object
ACTET_X           object
NOMVOI_X          object
NUMVOI_X          object
TYPEVOI_X         object
BISTER_X          object
CPLADR_X          object
CLT_C_C           object
CLT_X             object
DLT_X             object
PLT_X             object
DEPCOM_CODE       object
I_MCA_C           object
I_REPRISE_ACT     object
I_SIRET_M         object
NOTE_MCA_ADR_C    object
NOTE_MCA_NOM_C    object
PROFA_X           object
PROFI_X           object
PROFS_X           object
VARDOMPART_X      object
SIRET_DEC         object
SIRETC            object
SIRETM            object
id                object
ILT_X             object
code_retour       object
naf1              object
pct1              object
naf2              object
pct2              object
naf3              object
pct3              object
naf4              object
pct4              object
naf5              object
pct5              object


In [96]:
for x in dtypes_to_list(df).keys():
    print(x)

cabbi
typcol
depcom_code
rs_x
ilt_x
vardompart_x
numvoi_x
bister_x
typevoi_x
nomvoi_x
cpladr_x
actet_x
actet_c_c
actet_c
clt_c_c
clt_x
dlt_x
plt_x
siretc
note_mca_adr_c
note_mca_nom_c
i_mca_c
profi_x
profs_x
profa_x
i_reprise_act
i_siret_m
siret_dec
siretm
proces_codage_init
arbitrage
siretq
i_siret_q
siret_arb
x
y
sourcexyw
qual
siret_final
code_retour
naf1
pct1
naf2
pct2
naf3
pct3
naf4
pct4
naf5
pct5
nbtot
nb5


In [90]:
df.dtype

AttributeError: 'DataFrame' object has no attribute 'dtype'

In [11]:
df.dtypes

sirus_id                             int64
nic                                  int64
ape                                 object
apet                                object
eff_3112_et                        float64
eff_etp_et                         float64
eff_et_effet_daaaammjj             float64
enseigne_et1                        object
nom_comm_et                         object
adr_et_loc_geo                       int64
adr_et_compl                        object
adr_et_voie_num                    float64
adr_et_voie_repet                   object
adr_et_voie_type                    object
adr_et_voie_lib                     object
adr_et_cedex                       float64
adr_et_distsp                       object
sir_adr_et_com_lib                  object
adr_et_post                          int64
adr_et_l1                           object
adr_et_l2                           object
adr_et_l3                           object
adr_et_l4                           object
adr_et_l5  

In [3]:
dtypes_list = []
with fs.open(os.path.join(remote_data_dir,F'rp_final_2014.csv')) as f:
        for df in pd.read_csv(f,sep=';',chunksize=2000,engine='python',encoding='unicode_escape'):
            dtypes_list.append(df.dtypes)

In [4]:
dtypes_list_2017 = []
with fs.open(os.path.join(remote_data_dir,F'rp_final_2017.csv')) as f:
        for df in pd.read_csv(f,sep=';',chunksize=2000,engine='python',encoding='unicode_escape'):
            dtypes_list_2017.append(df.dtypes)

In [6]:
for index,item in enumerate(dtypes_list):
    for num_col,col in enumerate(dtypes_list_2017[0] == dtypes_list_2017[index]):
        if col == False:
            print(dtypes_list_2017[0].reset_index().iloc[[num_col]])
            print(" ")

In [8]:
for index,item in enumerate(dtypes_list):
    for num_col,col in enumerate(dtypes_list[0] == dtypes_list[index]):
        if col == False:
            print(dtypes_list[0].reset_index().iloc[[num_col]])
            print(" ")

In [13]:
dtypes_list[0]

CABBI                   int64
TYPCOL                  int64
DEPCOM_CODE            object
RS_X                   object
ILT_X                 float64
VARDOMPART_X          float64
NUMVOI_X              float64
BISTER_X               object
TYPEVOI_X              object
NOMVOI_X               object
CPLADR_X               object
ACTET_X                object
ACTET_C_C              object
ACTET_C                object
CLT_C_C                object
CLT_X                  object
DLT_X                  object
PLT_X                 float64
SIRETC                float64
NOTE_MCA_ADR_C        float64
NOTE_MCA_NOM_C        float64
I_MCA_C                object
PROFI_X                object
PROFS_X                object
PROFA_X                object
I_REPRISE_ACT          object
I_SIRET_M             float64
SIRET_DEC               int64
SIRETM                float64
PROCES_CODAGE_INIT     object
ARBITRAGE              object
SIRETQ                float64
I_SIRET_Q               int64
SIRET_ARB 

In [10]:
# dtypes_list[1]